# Speckle Tracking

In [ ]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal(Path("../../_static/sample_files/ideal_linear_pressure_increase copy.xplt"))
lv_typeA = get_lv_typeA(
    Path("../../_static/sample_files/lv_typeA_hex.vtk"),
    Path("../../_static/sample_files/sample_displacement_lv_typeA_hex_states_linear_press_incr.pbz2")
    )

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

## What are Speckles?

Speckles ...

In [ ]:
a = np.radians(0.0)
normal = [np.cos(a),np.sin(a),0.0]
print(normal)
n_subsets = 6
_ = lv_typeA.create_speckles(
            collection="SAMPLETEST2",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=2.5,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="z2",
            cluster_criteria="angles3",
            n_clusters=18,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLETEST2")

plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                       )

In [ ]:
normal = [0,0,1]
n_subsets = 6
_ = lv_typeA.create_speckles(
            collection="SAMPLETEST3",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=4.0,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="angles",
            cluster_criteria="angles3",
            n_clusters=18,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLETEST3")

plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                    #    k_center_filters=dict(
                    #        mfilter_ws=3, 
                    #        sfilter_ws=5, 
                    #        sfilter_or=1, 
                    #        keep_first=True, 
                    #        keep_last=True)
                       )

In [ ]:
from project_heart.utils.vector_utils import *
a = np.radians(90.0)
normal = np.array([np.cos(a),np.sin(a),0.0])
normal = np.array([0.0,0.0,1.0])

n_subsets = 6
n_cluster = 8
_ = lv_typeA.create_speckles(
            collection="SAMPLET15",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=4.5,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            # subsets_criteria="z2",
            subsets_criteria="angles",
            # cluster_criteria="z2",
            n_clusters=0,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLET15")
for i in range(n_subsets):
    
    spk = sample_spk_typeA[i]
    ref_center = spk.center
    n_subsets = 4
    pts = lv_typeA.points(mask=spk.ids)

    import logging
    logger = logging.getLogger("test")
    logger.setLevel(logging.DEBUG)
    logger.debug("i: {}.".format(i))
    
    if normal[2] < 0:
        normal = -normal
        logger.debug("using opposite normal for computation as negative "
                     "(on z) axis are not fully tested: {}.".format(normal))
    # project points onto a single plane based on spk normal
    logger.debug("pts.shape: {}.".format(pts.shape))
    logger.debug("pts[:5]: \n{}.".format(pts[:5]))
    plane_d = calc_plane_d(normal, ref_center)
    logger.debug("plane_d: {}.".format(plane_d))
    ppts = project_pts_onto_plane(pts, normal, plane_d)
    p_center = project_pts_onto_plane(ref_center, normal, plane_d)[0]
    logger.debug("p_center: {}.".format(p_center))
    # compute angles between a reference vector and other vectos
    vecs = ppts - p_center 
    vecs_sum = np.linalg.norm(vecs, axis=1)
    ref_max_pt = np.argmax(vecs_sum)
    ref_min_pt = np.argmin(vecs_sum)
    logger.debug("ref_max_pt: {}.".format(ref_max_pt))
    logger.debug("ref_max_pt: {}.".format(ref_min_pt))
    ref_max_vec = vecs[ref_max_pt]
    ref_min_vec = vecs[ref_min_pt]
    logger.debug("ref_max_vec: {}.".format(ref_max_vec))
    angles = angle_between(vecs, ref_max_vec, check_orientation=False) # initial guess
    min_a, max_a = np.min(angles), np.max(angles)
    logger.debug("min_a: {}.".format(np.degrees(min_a)))
    logger.debug("max_a: {}.".format(np.degrees(max_a)))
    min_idx, max_idx = np.argmin(angles), np.argmax(angles)
    ref_max_vec = vecs[max_idx]
    ref_min_vec = vecs[min_idx]
    logger.debug("ref_max_vec: {}.".format(ref_max_vec))
    logger.debug("ref_min_vec: {}.".format(ref_min_vec))
    # sort_ids = np.argsort(angles)
    min_a, max_a = np.min(angles), np.max(angles)
    logger.debug("min_a: {}.".format(np.degrees(min_a)))
    logger.debug("max_a: {}.".format(np.degrees(max_a)))
    # create bins
    bins = np.digitize(
        angles, np.linspace(min_a*0.999, max_a*1.001, n_cluster+1))
    # update reference vectors (make sure we have extreme pts)
    ref_last_bin_idx = np.argmax(bins)    
    ref_first_bin_idx = np.argmin(bins)
    ref_last_vec = vecs[ref_last_bin_idx]
    ref_first_vec = vecs[ref_first_bin_idx] 
    
    first_cross_last = np.cross(unit_vector(ref_first_vec), unit_vector(ref_last_vec)) 
    cross_normal = np.degrees(angle_between(normal, first_cross_last, check_orientation=False)) 
    first_last_angle = np.degrees(angle_between(ref_first_vec, ref_last_vec, check_orientation=True, zaxis=normal))
    
    logger.debug("normal: {}.".format(normal))
    logger.debug("first_cross_last: {}.".format(first_cross_last))
    logger.debug("cross_normal: {}.".format(cross_normal))
    logger.debug("first_last_angle: {}.".format(first_last_angle))
    angles = angle_between(vecs, vecs[max_idx], check_orientation=False)
    
    a_normal_Zaxis = np.degrees(angle_between(normal, [0.0,0.0,1.0], check_orientation=False))
    if normal[2] < 45:
        logger.debug("Re-computing bins as angle from normal to Z axis < 45: {}.".format(a_normal_Zaxis))
        min_a, max_a = np.min(angles), np.max(angles)
        logger.debug("min_a: {}.".format(np.degrees(min_a)))
        logger.debug("max_a: {}.".format(np.degrees(max_a)))
        bins = np.digitize(
            angles, np.linspace(min_a*0.999, max_a*1.001, n_cluster+1))
        
        
    centers = np.asarray([np.mean(ppts[bins==bin_id], axis=0) for bin_id in sorted(np.unique(bins))], dtype=np.int64)
    c_vecs = centers - p_center
    c_angles = np.degrees(angle_between(c_vecs, c_vecs[0], check_orientation=True, zaxis=normal))
    c_grads = np.gradient(c_angles[1:])
    mean_c_grads = np.mean(c_grads)
  
    logger.debug("mean_c_grads: {}.".format(mean_c_grads))
    
    
    # should_reverse = True
    should_reverse = True
    a_normal_Xaxis = np.degrees(angle_between(normal, [1.0,0.0,0.0], check_orientation=False))
    if a_normal_Xaxis <= 45:
    # if np.round(normal[0], 3) >= 0 and np.round(normal[1], 3) <= 0:
        should_reverse = not should_reverse
        # logger.debug("Flip should_reverse as normal[0] >= 0 and normal[1] <= 0: [{}].".format(should_reverse))   
        logger.debug("Flip should_reverse due normal orientation -> [{}]: a_normal_Xaxis: {}.".format(should_reverse, a_normal_Xaxis))   
    
    if cross_normal < 90 and mean_c_grads > 0:
        should_reverse = not should_reverse
        logger.debug("bins should reverse [{}] as cross_normal < 90 and mean_c_grads > 0.".format(should_reverse))
    elif cross_normal < 90 and first_last_angle > 180:
        should_reverse = not should_reverse
        logger.debug("bins should reverse [{}] as cross_normal < 0 and first_last_angle > 180.".format(should_reverse))
    elif cross_normal > 90 and first_last_angle > 180 and mean_c_grads > 0:
        should_reverse = not should_reverse
        logger.debug("bins should reverse [{}] as cross_normal > 0, first_last_angle > 180 and mean_c_grads > 0.".format(should_reverse))
        
    
    if should_reverse:
        logger.debug("reversing bins.")
        uvals = np.unique(bins)
        logger.debug("uvals: {}.".format(uvals))
        new_bins = np.zeros(len(bins), dtype=np.int64)
        for i, u in enumerate(uvals[::-1]):
            new_bins[bins==u] = i
        bins = new_bins + 1
      
    
    plotter = lv_typeA.plot_speckles(spk, 
                        cmap=None,
                        color="blue",
                        point_size=100,
                        add_k_centers=False, 
                        k_centers_as_line=True, 
                        k_bins=False,
                        re = True
                        )
    
    plotter.add_lines(np.vstack((p_center, p_center+5*normal)), color="magenta")
    plotter.add_lines(np.vstack((p_center, p_center+5*first_cross_last)), color="orange")
    
    
    plotter.add_lines(np.vstack((p_center, ppts[ref_max_pt])), color="blue")
    plotter.add_lines(np.vstack((p_center, ppts[ref_min_pt])), color="green")
    
    
    plotter.add_lines(np.vstack((p_center, ppts[ref_first_bin_idx])), color="cyan")
    plotter.add_lines(np.vstack((p_center, ppts[ref_last_bin_idx])), color="red")
    
    
    plotter.add_points(p_center, color="green", point_size=240)
    plotter.add_points(ppts, color="magenta", point_size=200)
    plotter.add_points(pts, scalars=bins-1, categories=True, cmap="hot", point_size=240)
    plotter.show(window_size=(600,400))

## Sample Speckles

### Longitudinal Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_long[1], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_long[0], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)

### Circumferential Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_circ[7], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_circ[7], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)